In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors

In [ ]:
# Предсказание цен на недвижимость в Калифорнии
# Скачан с https://www.kaggle.com/datasets/camnugent/california-housing-prices/data
dataset = pd.read_csv("housing.csv")
dataset

In [ ]:
# Пустые значения есть только в колонке 'total_bedrooms' - заполним их -1
dataset['total_bedrooms'] = dataset['total_bedrooms'].fillna(-1)

# Выделяем целевую переменную (медианная стоимость дома в ближайшем окружении)
feature_columns = ['longitude', 'latitude', 'housing_median_age', 'total_rooms','total_bedrooms', 'population', 'households', 'median_income','ocean_proximity']
target_column = 'median_house_value'
data = dataset[feature_columns]
target = dataset[target_column]

In [ ]:
# Обернем в функции код обучения и валидации качества моделей (линейной регрессии и random forest)
# Для оценки качества будем смотреть на MAPE (на сколько процентов в среднем ошиблись) и MAE (на сколько в среднем долларов оишблись)
def train_evaluate_lr(train_data, train_target, val_data, val_target):
    model = LinearRegression()
    scaler = StandardScaler()

    # Нормализуем данные перед обучением
    scaled_train = scaler.fit_transform(train_data)
    scaled_val = scaler.transform(val_data)

    model = model.fit(scaled_train, train_target)
    print("Модель обучена")

    val_pred = model.predict(scaled_val)
    mape = np.round(mean_absolute_percentage_error(val_target, val_pred) * 100, 2) # Переведем в проценты
    mae = int(mean_absolute_error(val_target, val_pred))
    print("MAPE:", mape, "%")
    print("MAE:", mae, "$")

    # Вычислим средний вклад фичи в предсказание - коэффициент в модели от фичи
    coef_dict = {feature: np.round(abs(model.coef_[i]), 2) for i, feature in enumerate(train_data.columns)}
    sorted_coefs = sorted(coef_dict.items(), key=lambda x: x[1], reverse=True)
    print("Вклад фичей в предсказание:")
    for feature, coef in sorted_coefs:
        print(feature, coef)


def train_evaluate_rf(train_data, train_target, val_data, val_target):
    model = RandomForestRegressor(max_depth=8)

    model = model.fit(train_data, train_target)
    print("Модель обучена")

    val_pred = model.predict(val_data)
    mape = np.round(mean_absolute_percentage_error(val_target, val_pred) * 100, 2) # Переведем в проценты
    mae = int(mean_absolute_error(val_target, val_pred))
    print("MAPE:", mape, "%")
    print("MAE:", mae, "$")

In [ ]:
# Бейзлайн без доп фичей (обучаем на чем есть)
baseline = data[[feature for feature in feature_columns if feature != "ocean_proximity"]]
train_data, val_data, train_target, val_target = train_test_split(baseline, target, test_size=0.2, random_state=42)
train_evaluate_lr(train_data, train_target, val_data, val_target)

In [ ]:
# Сделаем кластеризацию и получим номера кластеров для каждого объекта
kmeans_feats = data[[feature for feature in feature_columns if feature != "ocean_proximity"]]
train_data, val_data, train_target, val_target = train_test_split(kmeans_feats, target, test_size=0.2, random_state=42)
coord_columns = ["latitude", "longitude"]

# Как и в прошлый раз, расстояния считаем евклидово (можно по haversine)
n_clusters = 100
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
train_data["cluster"] = kmeans.fit_predict(train_data[coord_columns])
val_data["cluster"] = kmeans.predict(val_data[coord_columns])

# Для каждого кластера вычислим среднюю, минимальную и максимальную стоимость домов через groupby
# Не забываем, что все это надо вычислять на train чтобы не было лика
train_data["target"] = train_target
agg_prices = train_data[["target", "cluster"]].groupby("cluster").agg(["mean", "min", "max", "std"])
agg_prices.columns = agg_prices.columns.droplevel(0)
agg_prices = agg_prices.add_prefix("price_in_cluster_")

# Подсоединяем к нашим данным по кластеру
train_data = train_data.merge(agg_prices, on="cluster", how="left")
val_data = val_data.merge(agg_prices, on="cluster", how="left")

# Из KMeans можно вытащить центройды кластеров
centroids = kmeans.cluster_centers_
centroids_df = pd.DataFrame({"cluster": list(range(n_clusters)), "centroid_lat": list(centroids[:, 0]), "centroid_long": list(centroids[:, 1])})

# Посчитаем расстояние от объекта до центра кластера
train_data = train_data.merge(centroids_df, on="cluster", how="left")
train_data["distance_to_center"] = train_data[["latitude", "longitude", "centroid_lat", "centroid_long"]] \
    .apply(lambda row: np.sqrt((row["latitude"] - row["centroid_lat"])**2 + (row["longitude"] - row["centroid_long"])**2), axis=1)
val_data = val_data.merge(centroids_df, on="cluster", how="left")
val_data["distance_to_center"] = val_data[["latitude", "longitude", "centroid_lat", "centroid_long"]] \
    .apply(lambda row: np.sqrt((row["latitude"] - row["centroid_lat"])**2 + (row["longitude"] - row["centroid_long"])**2), axis=1)

# Убираем лишние колонки
train_data = train_data.drop(columns=["target", "cluster", "centroid_lat", "centroid_long"])
val_data = val_data.drop(columns=["cluster", "centroid_lat", "centroid_long"])
train_data_kmeans = train_data
val_data_kmeans = val_data

train_evaluate_lr(train_data_kmeans, train_target, val_data_kmeans, val_target)

In [ ]:
# А теперь посмотрим как добавление фичей влияет на Random Forest
print("Базовый вариант - без доп фичей")
train_data, val_data, train_target, val_target = train_test_split(baseline, target, test_size=0.2, random_state=42)
train_evaluate_rf(train_data, train_target, val_data, val_target)

print("С фичами c помощью KMeans")
train_evaluate_rf(train_data_kmeans, train_target, val_data_kmeans, val_target)

In [ ]:
# Давайте попробуем добавить признаки от соседей
kmeans_feats = data[[feature for feature in feature_columns if feature != "ocean_proximity"]]
train_data, val_data, train_target, val_target = train_test_split(kmeans_feats, target, test_size=0.2, random_state=42)

coord_columns = ["latitude", "longitude"]

# Предполагая что карта разделена на квадраты вида
# # #
# T #
# # #
# Попробуем найти 8 и 24 ближайших соседей (1ый и 2ой уровни близости)
num_neighbors = [8, 24]
for n_neighbors in num_neighbors:
    nn = NearestNeighbors(n_neighbors=n_neighbors)
    nn.fit(train_data[coord_columns])
    train_data["neighbors"] = list(nn.kneighbors(train_data[coord_columns], return_distance=False))
    val_data["neighbors"] = list(nn.kneighbors(val_data[coord_columns], return_distance=False))

    # Сделаем функцию для подсчета среднего таргета у соседей
    def avg_neigh_price(neigh_indexes):
        return train_target[neigh_indexes].mean()

    train_target = train_target.reset_index(drop=True)
    train_data[f"{n_neighbors}_neigh_avg_price"] = train_data["neighbors"].apply(avg_neigh_price)
    val_data[f"{n_neighbors}_neigh_avg_price"] = val_data["neighbors"].apply(avg_neigh_price)

train_data = train_data.drop(columns=["neighbors"])
val_data = val_data.drop(columns=["neighbors"])
train_data_neigh = train_data
val_data_neigh = val_data

train_evaluate_lr(train_data_neigh, train_target, val_data_neigh, val_target)

In [ ]:
# А теперь посмотрим как добавление фичей влияет на Random Forest
print("Базовый вариант - без доп фичей")
train_data, val_data, train_target, val_target = train_test_split(baseline, target, test_size=0.2, random_state=42)
train_evaluate_rf(train_data, train_target, val_data, val_target)

print("С фичами по соседям")
train_evaluate_rf(train_data_neigh, train_target, val_data_neigh, val_target)